# Predictive modeling of edX Learner Attrition

**Problem:** MOOC courses have notoriously high dropout rates and Microsoft courses are no exception. How can we understand dropout trends and do something about them to keep users engaged

**Solution:** Intervene with users before they drop out.

**How?** Predict whether a user in a course will drop out in the next week of the course and engage them with support resources.
* Aggregate user events to understand engagement paterns
* Determine drop out weeks for each user in a course
* Build a model to predict the determined dropout weeks
* Build a cloud service to email "at-risk" users with support resources

## 1 - Packages

In [71]:
import pymssql
import pandas as pd
import numpy as np
from textblob import TextBlob
from datetime import datetime, timedelta
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import cntk as C
from collections import Counter

## 2 - Let's look at a sample from our dataset

In [72]:
features = pd.DataFrame([
    {
        'user_id': 9999,
        'course_week': 4,
        'user_started_week': 3,
        'num_video_plays': 12.,
        'num_subsections_viewed': 23.,
        'num_problems_attempted': 8.,
        'num_problems_correct': 5.,
        'num_forum_posts': 1.,
        'num_forum_up_votes': 3,
        'avg_forum_sentiment': .72
    },
    {
        'user_id': 100001,
        'course_week': 4,
        'user_started_week': 0,
        'num_video_plays': 0.,
        'num_subsections_viewed': 0.,
        'num_problems_attempted': 0.,
        'num_problems_correct': 0.,
        'num_forum_posts': 0.,
        'num_forum_up_votes': 0,
        'avg_forum_sentiment': .0
    }
])

features[[
    'user_id', 'course_week', 'user_started_week', 'num_video_plays',
    'num_subsections_viewed', 'num_problems_attempted', 'num_problems_correct',
    'num_forum_posts', 'num_forum_up_votes', 'avg_forum_sentiment'
]]

,user_id,course_week,user_started_week,num_video_plays,num_subsections_viewed,num_problems_attempted,num_problems_correct,num_forum_posts,num_forum_up_votes,avg_forum_sentiment
0,9999,4,3,12.0,23.0,8.0,5.0,1.0,3,0.72
1,100001,4,0,0.0,0.0,0.0,0.0,0.0,0,0.00


## 3 - Pull data from SQL Server Staging and Data Warehouse
Let's start our pipeline by getting all the raw data from our SQL Data Warehouse

In [73]:
## Essential Statistics for Data Science in Excel (Current Run)
course_id = 'Microsoft+DAT222x+4T2017' 

In [74]:
def get_course_dates(course_dates_df):
    """
    Get the start and end dates for the course
    """
    def get_datetime_col(col_name):
        """
        Get column as a datetime object
        """
        return datetime.strptime(course_dates_df[col_name][0], '%Y-%m-%d')
    course_start_date = get_datetime_col('CourseRunStartDate')
    course_end_date = get_datetime_col('CourseRunEndDate')
    return (course_start_date, course_end_date)

In [75]:
events = pd.read_csv('./data/{}/events.csv'.format(course_id))
forums = pd.read_csv('./data/{}/forums.csv'.format(course_id))
course_starts = pd.read_csv('./data/{}/course_starts.csv'.format(course_id))
course_completions = pd.read_csv('./data/{}/course_completions.csv'.format(course_id))
course_dates = pd.read_csv('./data/{}/course_dates.csv'.format(course_id))
course_start_date, course_end_date = get_course_dates(course_dates)

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
(course_start_date, course_end_date)

(datetime.datetime(2017, 10, 17, 0, 0), datetime.datetime(2018, 1, 1, 0, 0))

In [77]:
events.head(3)

,user_id,date_key,event_type,event_source,course_id,event_grade,event_attempts,event_max_grade,event_sub_correct,event_time,course_week
0,6467324,20170905,play_video,browser,Microsoft+DAT222x+4T2017,0.0,0.0,0.0,0.0,2017-09-05T21:38:51.594573+00:00,-6
1,6467324,20170905,play_video,browser,Microsoft+DAT222x+4T2017,0.0,0.0,0.0,0.0,2017-09-05T21:40:03.604545+00:00,-6
2,6467324,20170905,play_video,browser,Microsoft+DAT222x+4T2017,0.0,0.0,0.0,0.0,2017-09-05T21:35:32.208874+00:00,-6


In [78]:
forums.head(3)

,title,comment_text,author_id,votes_up,votes_down,votes_count,votes_point,comment_count,parent_id,comment_thread_id,course_id,text_type,update_timestamp
0,NaN,"Hi, for question 1 : We have given the winning...",12639383,1,0,1,1,NaN,NaN,59f4887a50ef220a0e002976,Microsoft+DAT222x+4T2017,Comment,2017-10-30T14:25:39.308Z
1,NaN,"Hi, you have total of 500 potential customers ...",12639383,0,0,0,0,NaN,59f5c29b50ef2209de0029ce,59f0c2ab4ed74909c00026f7,Microsoft+DAT222x+4T2017,Comment,2017-10-30T14:17:53.148Z
2,NaN,"Hi, you need to know the outcomes to calculate...",12639383,0,0,0,0,NaN,NaN,59f627a94ed74909bd0029ec,Microsoft+DAT222x+4T2017,Comment,2017-10-30T14:02:25.739Z


In [79]:
course_starts.head(3)

,user_id,date_key
0,14868851,20170828
1,6467324,20170905
2,6654643,20170918


In [80]:
course_completions.head(3)

,user_id,date_key
0,14847381,20171011
1,16333494,20171011
2,14046171,20171019


## 4 - Let's start building our model's features

Remember our features from the beginning. We've also added the user's last_active_week and completed_week at this point

In [81]:
features = pd.read_csv('./data/{}/features.csv'.format(course_id))
features.tail(3)

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week
14532,16887541,6,0.0,0.0,0.0,5.0,NaN,NaN,NaN,6,6,-1
14533,16893011,6,0.0,0.0,0.0,5.0,NaN,NaN,NaN,6,6,-1
14534,16895134,6,12.0,17.0,13.0,13.0,NaN,NaN,NaN,6,6,-1


## 5 - The final Model Data with augmented samples

In the previous step there were gaps in our data. We only had samples for the weeks each user was active.
We need to represent the inactive weeks for each user.

At this step we are adding these inactive weeks as "negative" data points and the value we are looking to predict
**user_dropped_out_next_week**

In [82]:
model_data = pd.read_csv('./data/{}/model_data.csv'.format(course_id))
model_data.tail()

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_dropped_out_next_week
38267,16902396,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,-1.0,0
38268,16902396,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,-1.0,0
38269,16902396,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,-1.0,0
38270,16902396,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,-1.0,0
38271,16902396,5.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,-1.0,1


# 6 - Let's build a model (aka the fun stuff)

**Predicting user_dropped_out_next_week** 
* 0 = user will not drop out next week (GOOD!)
* 1 = user will drop out next week     (SEND HELP!)

In [84]:
from collections import Counter
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.utils import plot_model
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from pipeline.util import *

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', num_hidden_layers=2, hidden_layers_dim=8):
    # create model
    model = Sequential()
    model.add(Dense(hidden_layers_dim, input_dim=9, activation='relu'))
    for i in range(num_hidden_layers):
        model.add(Dense(hidden_layers_dim, activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


def get_data(past_course_ids, current_course_id):
    train = None
    for course_id_ in past_course_ids:
        course_run_data = pd.read_csv('./data/{}/model_data.csv'.format(course_id_))
        if train is None:
            train = course_run_data
        else:
            train = train.append(course_run_data)

    print('Training data done.')

    train = train.reset_index(drop=True)
    test = pd.read_csv('./data/{}/model_data.csv'.format(current_course_id))

    X_cols = [
        'course_week', 'num_video_plays', 'num_problems_attempted',
        'num_problems_correct', 'num_subsections_viewed', 'num_forum_posts',
        'num_forum_votes', 'avg_forum_sentiment', 'user_started_week',
    ]

    X_train = np.array(train[X_cols]).astype(np.float32)
    X_test = np.array(test[X_cols]).astype(np.float32)

    y_train = np.array(train['user_dropped_out_next_week']).astype(np.float32)
    y_test = np.array(test['user_dropped_out_next_week']).astype(np.float32)

    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    print('SAVING X_TEST SCALED:', course_id)
    pd.DataFrame(X_test, columns=X_cols).to_csv('./data/{}/x_scaled_test_data.csv'.format(course_id))
    print('DONE')

    return (X_train, y_train, X_test, y_test)


def fit_score_predict(course_id, from_checkpoint=False):

    # TODO Fix how this training data is sampled
    # e.g. bootstrap sampling of a random number of courses
    # to get a total of > 1 million training samples
    past_course_ids = []
    for i in range(3):
        past_course_ids.append('Microsoft+DAT206x+{}T2017'.format(i + 1))
        past_course_ids.append('Microsoft+DAT207x+{}T2017'.format(i + 1))
    past_course_ids.append('Microsoft+DAT206x+4T2017')


    print('GETTING DATA: ', past_course_ids)
    X_train, y_train, X_test, y_test = get_data(past_course_ids, course_id)
    print('Done.')

    # One hot encode labels
    y_train = keras.utils.to_categorical(y_train, num_classes=2)
    y_test = keras.utils.to_categorical(y_test, num_classes=2)
    batch_size = 20

    if from_checkpoint:
        model = load_model('model.h5')
    else:
        model = create_model()

        print('Fitting model')
        model.fit(X_train, y_train, epochs=10, batch_size=batch_size)
        print('Done')
        try:
            model.save('model.h5')
        except:
            print('FAILED TO SAVE MODEL')

    print('Evaluating model on data for course: {}'.format(course_id))

    score = model.evaluate(X_test, y_test, batch_size)

    preds = model.predict(X_test, batch_size)

    # HEAVILY penalize false negatives due to the data imbalance of 0 classes
    y_labels = y_test.argmax(axis=1)
    softmax_threshold = Counter(y_labels)[1] / (Counter(y_labels)[1] + Counter(y_labels)[0])

    final_preds = []
    for pred in preds:
        if pred[1] > softmax_threshold:
            final_preds.append(1)
        else:
            final_preds.append(0)

    print('Done')
            
    conf_matrix = metrics.confusion_matrix(y_test.argmax(axis=1), final_preds)

    tn, fp, fn, tp = conf_matrix.ravel()
    total = len(y_test)
    final_acc = (tn + tp) / total
    print('Accuracy after softmax', final_acc)

    test_data_orig = pd.read_csv('./data/{}/model_data.csv'.format(course_id))
    test_data_orig['predicted_user_dropped_out_next_week'] = final_preds

    pred_pivot = _create_pivot_table(test_data_orig, 'predicted_user_dropped_out_next_week')
    real_pivot = _create_pivot_table(test_data_orig, 'user_dropped_out_next_week')

    save_df_to_file(pred_pivot, 'predicted_dropouts', course_id, type='excel')
    save_df_to_file(real_pivot, 'real_dropouts', course_id, type='excel')
    save_df_to_file(test_data_orig, 'model_data_with_preds', course_id)

    print('Model Score before softmax: ', score)
    print('Accuracy after softmax', final_acc)
    print('CONFUSION MATRIX: ')
    print(conf_matrix)
    print(conf_matrix / len(y_test))

    return (final_preds, final_acc, conf_matrix)

def _create_pivot_table(df, val_col):
    df_pivot = df.pivot_table(
        index='user_id', columns=['course_week'], values=val_col, fill_value=-1
    )
    df_colored = df_pivot.style.applymap(_cell_colors)
    return df_colored

def _cell_colors(s):
    ret = 'background-color: {}'
    if s == 0:
        ret = ret.format('#228b22')
    elif s == 1:
        ret = ret.format('#dc143c')
    else:
        ret = ret.format('#d3d3d3')

    return ret

### Run the pretrained model...

In [85]:
final_preds, final_acc, conf_matrix = fit_score_predict(course_id, from_checkpoint=True)

GETTING DATA:  ['Microsoft+DAT206x+1T2017', 'Microsoft+DAT207x+1T2017', 'Microsoft+DAT206x+2T2017', 'Microsoft+DAT207x+2T2017', 'Microsoft+DAT206x+3T2017', 'Microsoft+DAT207x+3T2017', 'Microsoft+DAT206x+4T2017']
Training data done.
SAVING X_TEST SCALED: Microsoft+DAT222x+4T2017
DONE
Done.
Evaluating model on data for course: Microsoft+DAT222x+4T2017
  240/38272 [..............................] - ETA: 28s

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input5330") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


38272/38272 [==============================] - 10s 269us/step
Done
Accuracy after softmax 0.883570234114
Model Score before softmax:  [0.28691773203179788, 0.8608120959750486]
Accuracy after softmax 0.883570234114
CONFUSION MATRIX: 
[[25951  3633]
 [  823  7865]]
[[ 0.67806752  0.09492579]
 [ 0.02150397  0.20550272]]


**Expected model accuracy**:
<table> 
    <tr>
        <td> **Accuracy** </td>
        <td> 0.88357 </td>
    </tr>
</table>

**Expected Output of Confusion Matrix (Totals)**:
<table> 
    <tr>
        <td></td>
        <th> **Predicted user stayed in the course** </th>
        <th> **Predicted user dropped out of the course** </th>
    </tr>
    <tr>
        <td> **User stayed in the course** </td>
        <td> TN - 25951 </td>
        <td> FP - 3633 </td>
    </tr>
    <tr>            
        <td> **User dropped out of the course** </td>
        <td> FN - 823 </td>
        <td> TP - 7865 </td>
    </tr>
</table>

**Expected Output of Confusion Matrix (Percentages)**:
<table> 
    <tr>
        <td></td>
        <th> **Predicted user stayed in the course** </th>
        <th> **Predicted user dropped out of the course** </th>
    </tr>
    <tr>
        <td> **User stayed in the course** </td>
        <td> TN - 68% </td>
        <td> FP - 9% </td>
    </tr>
    <tr>            
        <td> **User dropped out of the course** </td>
        <td> FN - 2% </td>
        <td> TP - 21% </td>
    </tr>
</table>